<a href="https://colab.research.google.com/github/TK-brsq/Research/blob/main/SimSiam_by_SEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install spikingjelly

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler as lrs
from torch.utils.data import DataLoader, ConcatDataset
from torchvision.transforms import v2 as TF
from torchvision import datasets
'''
import spikingjelly
from spikingjelly import layer as jnn, neuron, functional as jF
'''
from tqdm import tqdm
import wandb

# Model

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, inplane, outplane, down_sampling=True):
        super(BasicBlock, self).__init__()
        self.down_sampling = down_sampling
        self.stride = 2 if down_sampling else 1
        self.down_sample  = jnn.Sequential(
            jnn.Conv2d(inplane, outplane, 3, 2, 1, bias=False),
            jnn.BatchNorm2d(outplane),
            neuron.IFNode()
        )

        layer = []
        layer.append(jnn.Conv2d(inplane, outplane, 3, self.stride, 1, bias=False))
        layer.append(jnn.BatchNorm2d(outplane))
        layer.append(jnn.IFNode())
        layer.append(jnn.Conv2d(outplane, outplane, 3, 1, 1, bias=False))
        layer.append(jnn.BatchNorm2d(outplane))
        layer.append(neuron.IFNode())
        self.layer = jnn.Sequential(*layer)

    def forward(self, x):
        identity = x
        x = self.layer(x)
        if self.down_sampling:
            identity = self.conv1x1(identity)
        x += identity
        return x

class ResNet18(nn.Module):
    def __init__(self, classes=10):
        super(ResNet18, self).__init__()

        self.first = nn.Sequential(
            jnn.Conv2d(3, 32, 3, 1, 1, bias=False),
            jnn.BatchNorm2d(32)
        )
        self.block1 = BasicBlock(32, 32, False)
        self.block2 = BasicBlock(32, 32, False)
        self.block3 = BasicBlock(32, 64, True)
        self.block4 = BasicBlock(64, 64, False)
        self.block5 = BasicBlock(64, 128, True)
        self.block6 = BasicBlock(128, 128, False)
        self.block7 = BasicBlock(128, 256, True)
        self.block8 = BasicBlock(256, 256, False)
        self.projector = nn.Sequential(
            jnn.AdaptiveAvgPool2d((1, 1)),
            jnn.Linear(256, 1024, bias=False),
            jnn.BatchNorm1d(1024),
            neuron.IFNode(),
            jnn.Linear(1024, 1024, bias=False)
        )

        for layer in self.modules():
            if isinstance(layer, jnn.Conv2d):
                jnn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
            elif isinstance(layer, nn.BatchNorm2d):
                nn.init.ones_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        x = self.first(x)
        x = self.block1(x) #[64, 32, 32] -> [64, 32, 32]
        x = self.block2(x)
        x = self.block3(x) #[64, 32, 32] -> [128, 16, 16]
        x = self.block4(x)
        x = self.block5(x) #[128, 16, 16] -> [256, 8, 8]
        x = self.block6(x)
        x = self.block7(x) #[256, 8, 8] -> [512, 4, 4]
        x = self.block8(x)
        x = self.last(x)
        return x

In [ ]:
class SimSiam(nn.Module):
    def __init__(self, encoder, indim=512):
        super(SimSiam, self).__init__()
        self.encoder = encoder
        self.projector = nn.Sequential(
            jnn.Linear(indim, indim//4, bias=False),
            jnn.ReLU(),
            jnn.Linear(indim//4, indim, bias=False)
        )

    def forward(self, xi, xj):
        hi = self.encoder(xi)
        hj = self.encoder(xj)
        zi = self.projector(hi)
        zj = self.projector(hj)
        return zi, zj

In [ ]:
class Classifier(nn.Module):
    def __init__(self, indim, classes):
        super(Classifier, self).__init__()
        self.layer = nn.Sequential(
            jnn.Linear(indim, classes, bias=False),
            neuron.IFNode()
        )
    def forward(self, x):
        y = self.layer(x)
        return y

In [ ]:
class NT_Xent(nn.Module):
    def __init__(self, batch_size, tau, device):
        super(NT_Xent, self).__init__()
        self.batch_size = batch_size
        self.tau = tau
        self.device = device
        self.mask = self.make_mask()
        self.cosine = nn.CosineSimilarity(dim=2)
        self.Xent = nn.CrossEntropyLoss()

    def make_mask(self):
        mask = torch.eye(2 * self.batch_size)
        ones = torch.ones(self.batch_size)
        mask = mask + torch.diag(ones, self.batch_size) + torch.diag(ones, -self.batch_size)
        return ~mask.bool()

    def forward(self, zi, zj):
        z = torch.cat((zi, zj), dim=0)
        similarity = self.cosine(z.unsqueeze(1), z.unsqueeze(0)) / self.tau

        sim_ij, sim_ji = torch.diag(similarity, self.batch_size), torch.diag(similarity, -self.batch_size)
        positive = torch.cat((sim_ij, sim_ji), dim=0).reshape(2*self.batch_size, 1)
        negative = similarity[self.mask].reshape(2*self.batch_size, -1)

        labels = torch.zeros(2*self.batch_size, dtype=torch.long).to(self.device)
        logits = torch.cat((positive, negative), dim=1)
        loss = self.Xent(logits, labels)

        return loss / 2

# Utils

In [ ]:
class DataAugmentation:
    def __init__(self):
        self.device = device
        color_jitter = TF.ColorJitter(0.8, 0.8, 0.8, 0.2)
        self.tf = TF.Compose([
            TF.RandomResizedCrop(32, (0.36, 1)),
            TF.RandomHorizontalFlip(p=0.5),
            TF.RandomApply([color_jitter], p=0.8),
            TF.RandomGrayscale(p=0.2),
            TF.ToImage(),
            TF.ToDtype(torch.float32, scale=True)
        ])

    def __call__(self, x):
        return self.tf(x), self.tf(x)

In [ ]:
def get_loader(data='cifar10', split='train', batch_size=128, DA=False):
    tf = DataAugmentation() if DA else TF.Compose([TF.ToImage(), TF.ToDtype(torch.float32, scale=True)])
    if data == 'cifar10':
        match split:
            case 'train':
                data = datasets.CIFAR10('./data', train=True, tranform=tf, download=True)
            case 'test':
                data = datasets.CIFAR10('./data', train=False, tranform=tf, download=True)
            case 'all':
                train = datasets.CIFAR10('./data', train=True, tranform=tf, download=True)
                test = datasets.CIFAR10('./data', train=False, tranform=tf, download=True)
                data = ConcatDataset([train, test])
    elif data == 'stl10':
        match split:
            case 'train':
                data = datasets.STL10('./data', split='train', tranform=tf, download=True)
            case 'test':
                data = datasets.STL10('./data', split='test', tranform=tf, download=True)
            case 'all':
                data = datasets.STL10('./data', split='unlabeled', tranform=tf, download=True)
    else:
        print(f'{data} is not supported >_<. cifar10 or stl10 is supported')
    loader = DataLoader(data, batch_size, shuffle=True, drop_last=True, num_workers=2)
    return loader

In [ ]:
def train_(loader, model, optimizer, scheduler, criterion, device):
    jF.reset_net(model)
    running_loss = 0
    correct = 0
    model.train()
    for data, target in tqdm(loader):
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        #xm.optimizer_step(optimizer)
        running_loss += loss.item()
        correct += (out.argmax(1) == target).sum().item()
    scheduler.step()
    return running_loss, correct

In [ ]:
def train_cl(loader, model, optimizer, scheduler, criterion, device):
    jF.reset_net(model)
    running_loss = 0
    model.train()
    for (x1, x2), _ in tqdm(loader):
        x1, x2 = x1.to(device), x2.to(device)
        optimizer.zero_grad()
        z1, z2 = model(x1, x2)
        #zi, zj = zi.to(device), zj.to(device)
        loss = criterion(z1, z2)
        loss.backward()
        optimizer.step()
        #xm.optimizer_step(optimizer)
        running_loss += loss.item()
    scheduler.step()
    return running_loss

In [ ]:
def train_in_cl(loader, encoder, classifier, optimizer, criterion, device):
    jF.reset_net(encoder)
    jF.reset_net(classifier)
    correct = 0
    encoder.eval()
    classifier.train()
    for data, target in loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            z = encoder(data)
        out = classifier(z)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        #xm.optimizer_step(optimizer)
        correct += (out.argmax(1) == target).sum().item()
    return correct

In [ ]:
def save_checkpoint(filename, model, optimizer, scheduler):
    checkpoint = {
        'model_sd': model.state_dict(),
        'optimizer_sd': optimizer.state_dict(),
        'scheduler_sd': scheduler.state_dict()
    }
    torch.save(checkpoint, f'{filename}.pth')

def load_checkpoint(filename, model, optimizer, scheduler):
    checkpoint = torch.load(f'{filename}.pth')
    model.load_state_dict(checkpoint['model_sd'])
    optimizer.load_state_dict(checkpoint['optimizer_sd'])
    scheduler.load_state_dict(checkpoint['scheduler_sd'])

# Main

In [ ]:
#instance
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch_xla.device()
#------------------------#
loader = get_loader('cifar10', split='train', batch_size=512, DA=True)
loader2 = get_loader('cifar10', split='test', batch_size=128, DA=False)
N = len(loader2.dataset)
#------------------------#
encoder = ResNet18().to(device)
#model = SimCLR(encoder, 512).to(device)
#classifier = Classifier(512, 10).to(device)
#------------------------#
optimizer = optim.Adam(model.parameters())
#optimizer2 = optim.Adam(classifier.parameters())
#scheduler1 = lrs.LinearLR(optimizer, start_factor=0.01, total_iters=8)
scheduler2 = lrs.CyclicLR(optimizer, base_lr=1e-4, max_lr=5e-4, step_size_up=5)
#scheduler = lrs.SequentialLR(optimizer, [scheduler1, scheduler2], milestones=[8])
#------------------------#
criterion = NT_Xent(512, 0.2, device)
criterion2 = nn.CrossEntropyLoss()

'''
wandb.login()
run = wandb.init(
    project = 'name',
    config = {
        'Architecture': 'x',
        'optim': 'Adam(1e-3)',
        'sche1': 'x',
        'sche2': 'x',
        'sche': 'x',
        'criterion': 'x',
        'Data': 'Cifar10',
        'else': 'x'
    }
)
'''
#train
start_epoch = 0
epochs = 8
jF.set_step_mode(model, 'm')
for epoch in range(start_epoch, epochs):
    loss = train_cl(loader, model, optimizer, scheduler, criterion, device)
    correct = train_in_cl(loader2, model.encoder, classifier, optimizer2, criterion2, device)
    #wandb.log({'loss': loss, 'acc': correct*100/N})
    print(f'Epoch: {epoch} | loss: {loss} | acc: {correct*100/N}%')

#wandb.finish()
#save_checkpoint('!!!', simclr, optimizer, scheduler)

In [ ]:
# Result
#